In [1]:
pip install implicit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit

In [3]:
# Loading the data from june to august
data_june_august = pd.read_csv('Purchases_june_august.csv', sep=',')
data_june_august

C:\Users\Evgeniy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,sessionCookie,product.item_sku,product.original_price,product.sales_price,product.qty,visitedProducts,userid,_ts
0,3d1d47e2-2ab8-4e9f-93f0-6bc31f4d0131,B055901,579.0,463.0,1.0,"B055901,B056520",NaN,1592762884
1,75743e72-31b4-4c64-8a19-06f344355900,B052944,NaN,128.0,1.0,"B036794,B036801,B036800,B052944,B052943,B05573...",NaN,1597653307
2,75743e72-31b4-4c64-8a19-06f344355900,B052943,NaN,71.0,1.0,"B036794,B036801,B036800,B052944,B052943,B05573...",NaN,1597653422
3,f0242ece-238f-4743-999c-b14d145c405f,B056531,349.0,262.0,1.0,"B052623,B052624,B052625,B052620,B052619,B05262...",NaN,1597653300
4,75743e72-31b4-4c64-8a19-06f344355900,B036794,NaN,71.0,2.0,"B036794,B036801,B036800,B052944,B052943,B05573...",NaN,1592762905
...,...,...,...,...,...,...,...,...
440681,ogmHGbkri0PpiOD5KBOIz,NaN,NaN,NaN,NaN,"B027698,B015250,B049882,B023657r,B027701,B0112...",NaN,1598399050
440682,ogmHGbkri0PpiOD5KBOIz,NaN,NaN,NaN,NaN,"B027698,B015250,B049882,B023657r,B027701,B0112...",NaN,1598399050
440683,ogmHGbkri0PpiOD5KBOIz,NaN,NaN,NaN,NaN,"B027698,B015250,B049882,B023657r,B027701,B0112...",NaN,1598399050
440684,NmB6WZJONvO3VKPhXiFO5,NaN,NaN,NaN,NaN,"B024821,B039823,B052705r,B057444r,B054714,B018...",856399.0,1598399750


In [4]:
data_june_august = data_june_august.drop(['sessionCookie', 
                  'product.original_price', 
                  'product.sales_price', 
                  'visitedProducts', 
                  '_ts'], axis = 1)
data_june_august

,product.item_sku,product.qty,userid
0,B055901,1.0,NaN
1,B052944,1.0,NaN
2,B052943,1.0,NaN
3,B056531,1.0,NaN
4,B036794,2.0,NaN
...,...,...,...
440681,NaN,NaN,NaN
440682,NaN,NaN,NaN
440683,NaN,NaN,NaN
440684,NaN,NaN,856399.0


In [5]:
data_june_august = data_june_august.rename(columns={'product.item_sku': 'item', 'product.qty': 'qty', 'userid': 'user'})
data_june_august

,item,qty,user
0,B055901,1.0,NaN
1,B052944,1.0,NaN
2,B052943,1.0,NaN
3,B056531,1.0,NaN
4,B036794,2.0,NaN
...,...,...,...
440681,NaN,NaN,NaN
440682,NaN,NaN,NaN
440683,NaN,NaN,NaN
440684,NaN,NaN,856399.0


In [6]:
data_june_august = data_june_august.dropna(subset = ['item', 'qty', 'user']).reset_index(drop = True)
data_june_august['user'] = data_june_august['user'].astype('int')
data_june_august['qty'] = data_june_august['qty'].astype('int')
data_june_august

,item,qty,user
0,B022109,1,240123
1,B045493,1,240123
2,B012964,1,240123
3,B027170,1,240123
4,B009429,1,371765
...,...,...,...
45556,B054019,1,677137
45557,B055417,1,341455
45558,B041547,1,293049
45559,B003896,1,529210


In [7]:
# Loading the data from february to april
data_feb_april = pd.read_csv('Purchases_feb_april.csv', sep=';')
data_feb_april = data_feb_april.drop(['Price', 'Datetime'], axis = 1)
data_feb_april = data_feb_april.rename(columns={'userID': 'user', 'item_sku': 'item'})
data_feb_april

,item,qty,user
0,B053359,1,731667
1,B053359,1,358434
2,B053359,1,617811
3,B053359,1,277891
4,B053359,1,591779
...,...,...,...
294692,B022826,5,419794
294693,B022826,5,419794
294694,B053803,4,713508
294695,B025508,4,698528


In [8]:
# Appending to dataframes
data = data_june_august.append(data_feb_april)
data

,item,qty,user
0,B022109,1,240123
1,B045493,1,240123
2,B012964,1,240123
3,B027170,1,240123
4,B009429,1,371765
...,...,...,...
294692,B022826,5,419794
294693,B022826,5,419794
294694,B053803,4,713508
294695,B025508,4,698528


In [9]:
# Aggragating
data = data.groupby(['item','user'], as_index=False)['qty'].sum()
data = data.sort_values(by=['user']).reset_index(drop = True)
data

,item,user,qty
0,B028758,0,1
1,B055832,0,1
2,SA000233,0,1
3,B004287,0,1
4,B029492,0,1
...,...,...,...
254110,B051330,784154,1
254111,B051903,784154,1
254112,SA000019,784270,1
254113,B050199,784311,1


In [10]:
# Finding users that have more than N purchases total
N = 10
data_users_with_many_purchases = data.groupby(['user']).item.agg('count').to_frame('Amount of purchases').reset_index()
data_users_with_many_purchases = data_users_with_many_purchases[data_users_with_many_purchases['Amount of purchases'].ge(N)].reset_index(drop=True)
data_users_with_many_purchases

,user,Amount of purchases
0,0,39
1,353,10
2,445,22
3,868,13
4,873,10
...,...,...
7117,783350,14
7118,783480,36
7119,783499,12
7120,783693,10


In [23]:
# Leaving only those users in the dataframe who have more than N purchases total
data = data.join(data_users_with_many_purchases.set_index('user'), on = 'user', how = 'right').\
    drop(['Amount of purchases'], axis = 1)
data

,item,user,qty,user_id,item_id
0,B028758,0,1,0,2843
176,SA000233,0,1,0,7977
205,B004287,0,1,0,229
219,B036828,0,1,0,3008
242,B055483,0,1,0,6692
...,...,...,...,...,...
138936,B048438,680162,1,6141,4847
139471,B055914,680162,1,6141,6861
145005,B024631,680162,2,6141,2056
146245,B055916,680162,1,6141,6863


In [24]:
# Finding items that have more than M purchases total
M = 5
data_items_with_many_purchases = data.groupby(['item']).item.agg('count').to_frame('Amount of purchases').reset_index()
data_items_with_many_purchases = data_items_with_many_purchases[data_items_with_many_purchases['Amount of purchases'].ge(M)].\
    reset_index(drop=True)
data_items_with_many_purchases

,item,Amount of purchases
0,B000028,8
1,B000031,13
2,B000051,36
3,B000052,115
4,B000053,6
...,...,...
8123,sB006711,50
8124,sB017631,9
8125,sB023516,35
8126,sBB01011,13


In [25]:
# Leaving only those items in the dataframe that have more than M purchases total
data = data.join(data_items_with_many_purchases.set_index('item'), on = 'item', how = 'right').\
    drop(['Amount of purchases'], axis = 1).reset_index(drop=True)
data

,item,user,qty,user_id,item_id
0,B028758,0,1,0,2843
1,B028758,7198,1,41,2843
2,B028758,7832,2,44,2843
3,B028758,19260,1,122,2843
4,B028758,22988,2,153,2843
...,...,...,...,...,...
154712,B027041,433076,1,3555,2385
154713,B027041,412786,1,3332,2385
154714,B027041,646571,1,5831,2385
154715,B027041,601494,2,5333,2385


In [14]:
# Create a numeric user_id and item_id column
data['user'] = data['user'].astype('category')
data['item'] = data['item'].astype('category')
data['user_id'] = data['user'].cat.codes
data['item_id'] = data['item'].cat.codes
data

,item,user,qty,user_id,item_id
0,B028758,0,1,0,2843
1,B028758,7198,1,41,2843
2,B028758,7832,2,44,2843
3,B028758,19260,1,122,2843
4,B028758,22988,2,153,2843
...,...,...,...,...,...
154712,B058073,705863,2,6336,7466
154713,B058073,731054,1,6544,7466
154714,B058073,755808,2,6846,7466
154715,B058073,761238,1,6902,7466


In [15]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['qty'].astype(float), (data['item_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['qty'].astype(float), (data['user_id'], data['item_id'])))

In [16]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=128, regularization=0.01, iterations=30)

In [17]:
# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 100
data_conf = (sparse_item_user * alpha_val).astype('double')

In [18]:
#Fit the model
model.fit(data_conf)

In [19]:
#------------------------------------------------------------------
# CREATE USER RECOMMENDATIONS BY USING FUNCTIONS BUILT IN IMPLICIT  
#------------------------------------------------------------------

user_id = 122

recommended = model.recommend(user_id, sparse_user_item)

items = []
scores = []

# Get item names from ids
for item in recommended:
    idx, score = item
    items.append(data.item.loc[data.item_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of items names and scores
recommendations = pd.DataFrame({'item': items, 'score': scores})

print(recommendations)

       item     score
0   B040055  1.046737
1   B028778  0.977342
2   B015249  0.841366
3   B021790  0.830669
4   B015250  0.817230
5   B046948  0.801175
6  SET00013  0.714333
7   B054018  0.665731
8   B054924  0.653975
9   B022371  0.646682


In [20]:
amount_of_users = data['user_id'].nunique()
amount_of_users

7122

In [21]:
# Evaluation of prediction quality
number_of_items_to_be_recommended = 10
amount_of_users = data['user_id'].nunique()
sum_of_all_scores = 0

for user_id in range(amount_of_users):
  
  # Determing products bought by user
  data_fact = data.loc[data['user_id'] == user_id].sort_values(by = ['qty', 'item'], ascending = False)
  data_fact = data_fact[data_fact['qty'].ge(1)].reset_index(drop=True).drop(['user', 'qty', 'user_id', 'item_id'], axis = 1).reset_index(drop=True)
  data_fact = data_fact['item']
  data_fact.tolist()

  # Determing top products recommended by the model
  recommended = model.recommend(user_id, sparse_user_item, filter_already_liked_items=False)
  items = []
  scores = []
  for item in recommended:
    idx, score = item
    items.append(data.item.loc[data.item_id == idx].iloc[0])
    scores.append(score)
  data_predict = pd.DataFrame({'item': items, 'score': scores})
  data_predict = data_predict['item'].tolist()

  # Calculating overlap between fact and prediction for every user
  data_fact = set(data_fact)
  data_predict = set(data_predict)
  overlap = data_fact & data_predict
  overlap_percentage = float(len(overlap)) / len(data_predict) * 100 
  overlap_percentage

  sum_of_all_scores = sum_of_all_scores + overlap_percentage

prediction_quality = sum_of_all_scores / amount_of_users
prediction_quality

87.36169615276607